### scrapy

In [4]:
import scrapy
import requests
from scrapy.http import TextResponse

- gmarket best 200데이터 수집

In [ ]:
# 1. scrapy 프로젝트 생성

In [1]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\day1-20220803T002955Z-001\code\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [2]:
!tree gmarket /f

폴더 PATH의 목록입니다.
볼륨 일련 번호가 00000005 7C6C:FC35입니다.
C:\USERS\USER\DAY1-20220803T002955Z-001\CODE\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


In [10]:
# 2. xpath 찾기
url = 'http://corners.gmarket.co.kr/Bestsellers'
request = requests.get(url)
response = TextResponse(request.url, body=request.text, encoding='utf-8')

# links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li[1]/div[1]/a')
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
print(links[:2])

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953128102552122',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953128102552122']

In [11]:
link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding='utf-8')

title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()

print(title, price)

브라운 면도기 세정액 CCR5+1  25,030


In [ ]:
# 3. items.py 코드 작성 | model(데이터 구조) <-- 수집할 데이터의 칼럼 정의

In [12]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item:
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [ ]:
# 4. spider.py 코드 작성 | crawling 절차 정의

In [15]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Overwriting gmarket/gmarket/spiders/spider.py


In [ ]:
# 5. scrapy 프로젝트 실행

In [ ]:
# scrapy crawl GMB -o items.csv # 터미널에서 실행해야함

In [14]:
%pwd

'C:\\Users\\User\\day1-20220803T002955Z-001\\code'

In [16]:
pd.read_csv("gmarket/items.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'gmarket/items.csv'